In [1]:
#Install required versions
#pip install tensorflow==2.12
#pip install numpy==1.26.4
#pip install opencv-python matplotlib

In [2]:
#dataset with_ mask &without_ mask

#Full training code-MobileNetV2

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [4]:
#Dataset Path
dataset_path = "dataset/"
img_size = 128

In [5]:
#Data Augmentation
datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [6]:
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode="binary",
    subset="training"
)

Found 3077 images belonging to 2 classes.


In [7]:
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=32,
    class_mode="binary",
    subset="validation"
)

Found 769 images belonging to 2 classes.


In [8]:
#load MobileNetV2 Base Model
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(img_size, img_size,3)
)
base_model.trainable = False  #Freeze layers

In [9]:
#build custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.4)(x)
output = Dense(1, activation="sigmoid")(x)
model = Model(inputs=base_model.input, outputs=output)

In [10]:
model.compile(
    optimizer=Adam(1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [11]:
#Train Model
model.save("face_mask_mobilenetv2.h5")
print("Model saved")

Model saved


In [12]:
#convert to Tensorflow lite - Android
#run this after training
from tensorflow import keras
model = keras.models.load_model("face_mask_mobilenetv2.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]    #Smiler + faster model
tflite_model = converter.convert()
with open("face_mask_mobilenetv2.tflite", "wb") as f:
    f.write(tflite_model)
print("TFLite model saved!")


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpxfmtt337\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpxfmtt337\assets


TFLite model saved!


In [13]:
#this creates
#face_mask_mobilenetv2.tflite

In [14]:
##part3-Android App code (Kotlin/TFLite)
#Add TFLite dependency in build.gradle

In [15]:
#Gradle

#implementation
'org.tensorflow:tensorflow-lite:2.12.0'
#implementation
'org.tensorflow:tensorflow-lite-support:0.4.3'
#implementation 
'org.tensorflow:tensorflow-lite-metadata:0.4.3'


'org.tensorflow:tensorflow-lite-metadata:0.4.3'

In [16]:
#python code-load & run Face Mask Detection(Webcam)
import cv2
import numpy as np
import tensorflow as tf

In [17]:
#load trained model
from tensorflow import keras
model = keras.models.load_model("face_mask_mobilenetv2.h5")

In [18]:
#load face detector
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
img_size = 128

In [ ]:
#start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

        gray = cv2.cvtColor(frame, cv2.color_BGRGRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        for(x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]
            face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
            face = face / 128
            face = np.reshape(face, (1, IMG_SIZE, IMG_SIZE< 3))

            prediction = model.predict(face)[0][0]

            if prediction < 0.5:
                label = "Mask"
                color = (0, 255, 0)
            else:
                label = "No Mask"
                color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(
            frame, label, (x, y-10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2
        )
        cv2.imshow("Face Mask Detection", frame)
        

        if cv2.waitkey(1) & 0xFF == ord("q"):
             break

cap.release()
cv2.destroyAllWindows()
    
        

In [ ]:
#Run on a single Image 
img = cv2.imread("test.jpg")
img_resized = cv2.resize(img, (128, 128))
img_rescaled = img_resized / 255.0
img-input = np.reshape(img_rescaled, (1, 128, 128, 3))

result = model.predict(img_input)[0][0]

print("Mask" if result < 0.5 else "No Mask")